DO WE REPEAT ALL EXPERIMENTS IN PI? OR ONLY THE SECOND PI PART?

TODO: average error of vector? (k-fold cross validation applied to vector valued function...)

-> benchmarked whole experiment not just second run...

graphs all sufficiently explained?

MLE EXISTS???

# Workflow with results and statistics

In [ ]:
import numpy as np
import pandas as pd
import os
import plotly.graph_objects as go

os.chdir('/Users/nicopalm/Documents/Projects/ode/')

from src.minimizer.minimizer_library.differential_evolution import DifferentialEvolution

In [ ]:
minimizer = DifferentialEvolution()

## A priori model definition

According to the above experimental setup, given an experiment $\xi=(x_1,...,x_k)$ with experimental designs $$x_i=(SoC_i,T_i)\in X$$ we obtain the statistical model 
$$((\mathbb{R}^6)^k,\mathcal{B}^{6k},\mathcal{N}((f_\theta(T_i,SoC_i,t_i)_{i=1,...,6k},\sigma^2\mathbb{1}_{6k})):\theta\in \Theta)$$
with parameter space $$\Theta=(0,10)\times(0,10000)\times (0,1)$$ at (simulated) measurement times $t_i$ accordingly.
Observe that all statistical models inherit the same parameter space $\Theta$. Experimentally determining the (global) parameter $\theta$ calibrates the aging model $f_\theta$.

In [ ]:
from src.statistical_models.statistical_model_library.gaussian_noise_model import GaussianNoiseModel
from src.parametric_function_library.aging_model import AgingModel

In [ ]:
sigma = 0.002

theta = np.array([4, 2300, 0.8])

In [ ]:
lower_bounds_x = np.array([0.0001, 279.15])
upper_bounds_x = np.array([1, 333.15])

lower_bounds_theta = np.array([0.01, 0.01, 0.01])
upper_bounds_theta = np.array([10, 10000, 1])

In [ ]:
parametric_function = AgingModel()

def blackbox_model(x):
    return statistical_model.random(theta=theta, x=x)

In [ ]:
statistical_model = GaussianNoiseModel(function=parametric_function,
                                       lower_bounds_x=lower_bounds_x,
                                       upper_bounds_x=upper_bounds_x,
                                       lower_bounds_theta=lower_bounds_theta,
                                       upper_bounds_theta=upper_bounds_theta,
                                       sigma=sigma)

## Initial design of experiment

In [ ]:
from src.designs_of_experiments.design_library.latin_hypercube import LatinHypercube
from src.visualization.plotting_functions import *

Initially, we assume no prior knowledge about the value of model parameter $\theta$ to exist and an ability to conduct an experiment consisting of $k=5$ experimental designs.

In [ ]:
number_designs = 5

Therefore, the initial experiment is designed by the Latin Hypercube approach with ten samples out of $X$ resulting in the experiment $$\xi_{LH}=((T_1,SoC_1),...,(T_{10},SoC_{10})).$$

In [ ]:
LH = LatinHypercube(lower_bounds_design=lower_bounds_x,
                    upper_bounds_design=upper_bounds_x,
                    number_designs=number_designs)

The below figure indicates the calculated LH experiment with each blue dot representing an individual experimental design.

In [ ]:
data = [dot_scatter(x_dots=LH.design.T[0]*100, y_dots=LH.design.T[1], fill=None)]

fig0 = styled_figure(title=LH.name,
                     data=data,
                     title_x="State of charge [%]",
                     title_y="Temperature [K]")
fig0

## Conduct experiments

In [ ]:
from src.benchmarking.benchmarking import Benchmarking

We conduct the Latin Hypercube experiment $\xi_{LH}.$
This results in $30$ experimental data points $y_{i,j}$ based on measurements for $i=1,...,5$ experimental designs measured at $j=0,...,5$ points in time. 

In [ ]:
evaluation_initial_design = np.array([blackbox_model(x) for x in LH.design])

## Model Calibration

(MLE exists?)

The statistical model joining all experiments is given by
$$(\mathbb{R}^{30},\mathcal{B}^{30},\mathcal{N}((f_\theta(T_i,SoC_i,t_j)_{i=1,...,5,j=0,...,5},\sigma^2\mathbb{1}_{30})).$$
According to example \ref{FI example} the maximum likelihood estimate for $\theta$ of the experimental output $(y_{i,j})_{i=1,...,5,j=0,...,5}$ is given by the least square error estimate
$$\underset{\theta}{argmin}\, \sum_{i=1}^{5}\sum_{j=0}^5(y_{i,t}-f_\theta(T_i,SoC_i,t_j))^2.$$

In [ ]:
initial_theta = statistical_model.calculate_maximum_likelihood_estimation(
    x0=LH.design, y=evaluation_initial_design, minimizer=minimizer)

print("The resulting initial parameter estimate is then given by  \n",
      initial_theta)
print("(Reminder) The real theta is \n", theta)

Note that this parameter estimate is non-deterministic.
Bootstrapping with $N_{rep}=1000$ fold resampling within our 30-dimensional sample space allows us to quantify the statistical distribution of the parameter estimator.


In [ ]:
number_of_evaluations = 1000

In [ ]:
experiments = [LH]

benchmarking_LH = Benchmarking(blackbox_model=blackbox_model,
                               statistical_model=statistical_model,
                               designs_of_experiments=experiments)

benchmarking_LH.evaluate_designs(number_of_evaluations=number_of_evaluations,
                                 minimizer=minimizer)

The below figures show the normalized histograms (blue) of the estimated parameter vector component distributions after initial LH DoE as obtained via bootstrapping. The numbers are scaled evenly with the true parameter as the center and a range of $\pm 40\%$ around this value. The probability density function (pdf) of normal distributions with mean values defined by the true parameter values $\theta_i$ and variances by respective diagonal entries of the CRLB is shown in red within the according sub-figures. Note that bootstrapping is not part of the regular workflow but serves as additional validation in this particular example. 

In [ ]:
MLEs_LH_experiment = benchmarking_LH.maximum_likelihood_estimations[LH]

diagonal_CRLB_LH_experiment = statistical_model.calculate_cramer_rao_lower_bound(
    x0=LH.design, theta=theta).diagonal()

for index in range(3):
    data = [
        go.Histogram(
            x=MLEs_LH_experiment.T[index].T,
            histnorm='probability density',
            name="MLE",
        )
    ]
    fig = styled_figure_latex(data=data,
                              title_x=fr"$\theta_{index}$",
                              title_y=r"$\rho$")

    fig.add_trace(
        normal_distribution(x_range=np.arange(0.6 * theta[index],
                                              1.4 * theta[index], 0.001),
                            mu=theta[index],
                            sigma=np.sqrt(diagonal_CRLB_LH_experiment[index]),
                            name_dist="CRLB"))
    fig.show()

## Model Evaluation

In [ ]:
from src.metrics.metric_library.k_fold_cross_validation import KFoldCrossValidation
from src.metrics.error_functions.average_error import AverageError
from src.designs_of_experiments.design_library.pi_design import PiDesign

To evaluate the predictive ability of the now initially calibrated model, we choose the mean absolute error (MAE) as a performance measure determined by the leave-one-out cross-validation (LOOCV) method. This indicator is suitable for situations where only little data is available. 


In [ ]:
k_fold_validation = KFoldCrossValidation(statistical_model=statistical_model,
                                         minimizer=minimizer,
                                         error_function=AverageError(),
                                         number_splits=number_designs)

initial_cross_validation_value = k_fold_validation.calculate(
    design=LH, evaluations_blackbox_function=evaluation_initial_design)

print(
    f"The initial 10-fold cross-validation error is {initial_cross_validation_value}"
)

Observe this error to be in the range of the known measurement error $\sigma=0.002$.

The determinant of the FIM corresponding to 
$$(\mathbb{R}^{30},\mathcal{B}^{30},\mathcal{N}((f_\theta(T_i,SoC_i,t_i)_{i=1,...,30},\sigma^2\mathbb{1}_{30}))$$
is given by the below.


In [ ]:
det_FI = statistical_model.calculate_determinant_fisher_information_matrix(
    x0=LH.design, theta=initial_theta)
print(
    f"The determinant of the Fisher information matrix at the initial theta and design is \n{det_FI}"
)

In particular, the FIM $I(\theta_{init},\xi_{LH})$ is invertible.

The CRLB at the initially estimated parameter $\theta_{init}$ is given by 

In [ ]:
print("CRLB of pi-experiment at initial theta: \n",statistical_model.calculate_cramer_rao_lower_bound(x0=LH.design,
                                                       theta=initial_theta))


quantifying the relative expected standard deviation $\sigma_{rel,i}$
for each parameter component $\theta_{init,i}$ with respect to the initial parameter estimate $\theta_{init}$.

In [ ]:
print(
    'The relative expected standard deviations of the parameter estimators are \n',
    np.sqrt(
        statistical_model.calculate_cramer_rao_lower_bound(
            x0=LH.design, theta=initial_theta).diagonal()) / initial_theta)

## Prediction quality satisfactory?

In our example, we request a relative expected standard deviation below five percent (i.e. confidence level). 
According to 

In [ ]:
initial_relative_stds = np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(
        x0=LH.design, theta=initial_theta).diagonal()) / initial_theta

styled_figure(
    data=[
        go.Bar(x=[r"$\theta_{0}$",r"$\theta_{1}$",r"$\theta_{2}$"],
               y=initial_relative_stds,
               name="initial relative standrad deviations")
    ],
    title="Relative standard deviation of initial design for each parameter")

we accept estimation of $\theta_1$ and $\theta_2$ while refusing quality of $\theta_0$.   

However, even at this point we recognize the LOOCV error to be in the range of the measurement error. This reflects a suitable choice of the underlying statistical model (i.e. the aging model).
This is not surprising since the underlying simulation of experiments is given by $f_\theta$.



## pi-OED DoE

Up to this point, we have given the experiment $\xi_{LH}$ with its experimental results $(y_1,...,y_{5})$ and estimated the initial parameter $\theta_{init}$ from it. To reduce the uncertainty of the estimation of the first parameter component $\theta_0$, we conduct the pi-OED calculation with ten new experimental designs $x_{6},...,x_{10}$ granted. This corresponds to finding a global solution of the minimization problem
$$\underset{x_{6},...,x_{10}}{argmin}\, I^{-1}(\hat{\theta},( x_{1},...,x_{10}))_{11}.$$

In [ ]:
parameter = 0
pi_design = PiDesign(number_designs=number_designs,
                     lower_bounds_design=lower_bounds_x,
                     upper_bounds_design=upper_bounds_x,
                     column=parameter,
                     row=parameter,
                     initial_theta=initial_theta,
                     statistical_model=statistical_model,
                     previous_design=LH,
                     minimizer=minimizer)

The below figure shows the resulting new pi-OED experimental plan for designs $\xi_{pi}=(x_{6},...,x_{10})$. Number of repetitions is indicated within the graph.


In [ ]:
rounded_pi_design = np.round(pi_design.design[number_designs:], decimals=3)

data = [
    dot_scatter(x_dots=rounded_pi_design.T[0] * 100,
                y_dots=rounded_pi_design.T[1],
                fill=None)
]

fig = styled_figure(title=pi_design.name,
                    data=data,
                    title_x="State of charge [%]",
                    title_y="Temperature [K]")

unique_pi_values = np.unique(rounded_pi_design, axis=0, return_counts=True)

for i, _ in enumerate(unique_pi_values):
    fig.add_annotation(
        x=unique_pi_values[0].T[0][i] * 100,
        y=unique_pi_values[0].T[1][i],
        text=str(unique_pi_values[1][i]) + "-times",
        yshift=10,
        xshift=-4,
    )

fig

Adding the pi-OED calculated new experiment $\xi_{pi}=(x_{6},...,x_{10})$ to the already existing $\xi_{LH}$, the new CRLB calculated at $\theta_{init}$ is then given by

In [ ]:
print("CRLB of pi-experiment at initial theta: \n",statistical_model.calculate_cramer_rao_lower_bound(x0=pi_design.design,
                                                       theta=initial_theta))


This approximates the expected relative standard deviations (prior executing the new experimental designs) by the following.

In [ ]:
optimized_relative_stds = np.sqrt(
    statistical_model.calculate_cramer_rao_lower_bound(
        x0=pi_design.design, theta=initial_theta).diagonal()) / initial_theta


print("approximates relative standard deviations: \n",optimized_relative_stds)


The below bar chart shows the expected relative standard deviation at the initial theta of the initial (LH) experiments (blue) and the pi-experiment (red) indicating the decrease of the parameter uncertainty in the parameters. 

In [ ]:
styled_figure(
    data=[
        go.Bar(x=[r"$\theta_{0}$",r"$\theta_{1}$",r"$\theta_{2}$"],
               y=initial_relative_stds,
               name="initial relative standrad deviations"),
        go.Bar(x=[r"$\theta_{0}$",r"$\theta_{1}$",r"$\theta_{2}$"],
               y=optimized_relative_stds,
               name="optimized relative standard deviations")
    ],
    title="Relative standard deviations of initial and optimized design for each parameter"
)

## $2^{nd}$ run of  experiment and model calibration

MLE EXISTS???

After execution of the above calculated pi-OED experiment with $5$ experimental designs (yielding $5$ times $6$ additional data points), the corresponding statistical model reflecting the joint experiments of LH-DoE and pi-OED $\xi=(x_1,...,x_{10})$ is given by
$$(\mathbb{R}^{60},\mathcal{B}^{60},\mathcal{N}((f_\theta(T_i,SoC_i,t_j)_{i=1,...,10,j=0,...,5},\sigma^2\mathbb{1}_{60})).$$

Calculating the maximum likelihood estimation yields the optimized parameter estimation
\begin{align*}
    \theta_{opt} &= \underset{\theta}{argmin}\, \sum_{i=1}^{10}\sum_{j=0 }^5(y_{i,j}-f_\theta(T_i,SoC_i,t))^2.
\end{align*}

In [ ]:
evaluation_pi_design = np.array(
    [blackbox_model(x) for x in pi_design.design[number_designs:]])

In [ ]:
optimized_theta = statistical_model.calculate_maximum_likelihood_estimation(
    x0=pi_design.design,
    y=np.concatenate((evaluation_initial_design, evaluation_pi_design),
                     axis=0),
    minimizer=minimizer)

In [ ]:
print("optimized theta: \n", optimized_theta)

Again, bootstrapping with $N_{rep}=1000$ fold resampling within our (now 60-dimensional) sample space allows us to quantify the statistical distribution of our parameter estimator.
Resulting histograms of the parameter estimations are shown in chapter 2.1.

## $2^{nd}$ model evaluation

In [ ]:
optimized_cross_validation_value = k_fold_validation.calculate(
    design=pi_design,
    evaluations_blackbox_function=np.concatenate(
        (evaluation_initial_design, evaluation_pi_design), axis=0))

print('The cross validation value of the optimized design is',
      optimized_cross_validation_value[0],".")

Observe that this error is again in the range of the known measurement error $\sigma=0.002$.
The determinant of the corresponding FIM at $\theta_{opt}$ turns out to be greater than zero 

In [ ]:
det_optimized_FI = statistical_model.calculate_determinant_fisher_information_matrix(
    x0=pi_design.design, theta=optimized_theta)
print(
    f"The determinant of the Fisher information matrix at the optimized theta and design is \n{det_optimized_FI}"
)

yielding the following CRLB at $\theta_{opt}$.  

In [ ]:
print(f"CRLB of pi-experiment at optimized theta: \n",statistical_model.calculate_cramer_rao_lower_bound(x0=pi_design.design,
                                                       theta=optimized_theta))

Accordingly, the relative expected standard deviation
for each parameter component $\theta_{opt,i}$ with respect to the optimized parameter estimate $\theta_{opt}$ is shown in the below array.

In [ ]:
print(
    'The relative expected standard deviations of the parameter estimators is now \n',
    np.sqrt(
        statistical_model.calculate_cramer_rao_lower_bound(
            x0=pi_design.design, theta=optimized_theta).diagonal()) /
    optimized_theta)

## Prediction quality satisfactory?

Since all approximate relative standard deviations are below five percent, we accept the parameter quality.
Since the LOOCV MAE is in the range of the measurement error we accept the model quality and end our experiments.


# Benchmarking

In [ ]:
from src.designs_of_experiments.design_library.d_design import DDesign
from src.designs_of_experiments.design_library.random import Random
from ipywidgets import interact

In this paper, we claim our proposed pi-OED methodology effectively minimizes a chosen individual parameter's variance supporting an unbiased estimator approximating the true model parameter.
Our main objects of interest, therefore, are the
1. mean of the parameter and
2. variance of individual parameter (component)
estimations. 

In addition to the theoretical guarantees given in \ref{sec:methodology} we provide a benchmarking of pi-OED in order to strengthen our claim. Specifically, we compare the (estimated) mean and variances of the above experiment with those of three other experiments as calculated by
1. __Random Sampling__: we draw 20 experimental designs uniformly distributed over $X$.


In [ ]:
random_design = Random(number_designs=2 * number_designs,
                       lower_bounds_design=lower_bounds_x,
                       upper_bounds_design=upper_bounds_x)

2. __Latin-Hypercube Sampling__: we calculate a Latin-Hypercube experiment with 20 experimental designs.


In [ ]:
LH_full = LatinHypercube(lower_bounds_design=lower_bounds_x,
                         upper_bounds_design=upper_bounds_x,
                         number_designs=2 * number_designs)

3. __D-optimal DoE__: we replace the pi-OED in the above workflow with the D-optimal design of experiment (maximizing the FIM determinant) proposed by Atkinson and Donev \cite{atkinson_optimum_1992} at $\theta_{init}$ with five experimental designs.



In [ ]:
max_det = DDesign(number_designs=number_designs,
                  lower_bounds_design=lower_bounds_x,
                  upper_bounds_design=upper_bounds_x,
                  initial_theta=initial_theta,
                  statistical_model=statistical_model,
                  previous_design=LH,
                  minimizer=minimizer)

In [ ]:
experiments = [random_design, LH_full, pi_design, max_det]

In [ ]:
# plot design points
data = [
    dot_scatter(x_dots=experiments[0].design.T[0]*100,
                y_dots=experiments[0].design.T[1],
                fill=None)
]
fig0 = go.FigureWidget(data=data)
fig0 = update_layout_of_graph(fig0, title='Design points',
                             title_x="State of charge [%]",
                        title_y="Temperature [K]")


@interact(index=range(len(experiments)))
def update(index='LH'):
    with fig0.batch_update():
        fig0.update_layout(title=experiments[index].name)
        fig0.data[0].x = experiments[index].design.T[0]*100
        fig0.data[0].y = experiments[index].design.T[1]

    return fig0

For each of the above defined experiments, we bootstrap $N=1000$ experimental evaluations to calculate their according parameter estimate $\hat{\theta}^j (j=1,...,N)$ distributions. 



In [ ]:
benchmarking = Benchmarking(blackbox_model=blackbox_model,
                            statistical_model=statistical_model,
                            designs_of_experiments=experiments)

In [ ]:
benchmarking.evaluate_designs(number_of_evaluations=number_of_evaluations,
                              minimizer=minimizer)

In [ ]:
benchmarking.plot_estimations()

We estimate the mean of the parameter estimator by
\begin{align}
    M = \frac{1}{N}\sum_{j=1}^{N}\hat{\theta}^j
\end{align}
and the variance of the parameter component estimators $\hat{\theta}_i^j$ by
\begin{align}
    V_i = \frac{1}{N-1}\sum_{j=1}^{N}(M_j-\hat{\theta}_i^j)^2
\end{align}
for $i=1,2,3$.
Note that those are both best unbiased estimators assuming that the underlying distribution is a normal distribution. 

For each experiment we compare its parameter estimator's
1. estimated mean $M$.
2. relative deviation from the true parameter value
    $$M_{rel,i}=\frac{M_i-\theta_{true,i}}{\theta_{true,i}}$$ in percent.
3. estimated standard deviations $\sqrt{V_i}.$
4. approximate standard deviation at true parameter $\theta_{true}$ 
    $$\hat{\sigma}_{i}=\sqrt{I^{-1}(\theta_{true})_{ii}}.$$
5. relative deviation of approximate standard deviation at true parameter from the estimated standard deviation
    $$\hat{\sigma}_{rel,i}=\frac{\hat{\sigma}_{i}-\sqrt{V_j}}{\sqrt{V_j}}$$ in percent.



In [ ]:
from src.metrics.metric_library.std_parameter_estimations import StdParameterEstimations
from src.metrics.metric_library.determinant_of_fisher_information_matrix import DeterminantOfFisherInformationMatrix
from src.metrics.metric_library.estimation_mean_error import EstimationMeanError
from src.metrics.metric_library.estimation_mean_parameter_estimations import EstimationMeanParameterEstimations
from src.metrics.metric_library.estimation_variance_parameter_estimations import EstimationVarianceParameterEstimations

In [ ]:
estimation_mean_parameter_estimation = EstimationMeanParameterEstimations()

In [ ]:
estimated_means = []

for experiment in experiments:
    estimated_mean = estimation_mean_parameter_estimation.calculate(
        estimations_of_parameter=benchmarking.
        maximum_likelihood_estimations[experiment], )

    estimated_means.append(estimated_mean)

estimated_means = np.array(estimated_means)
relative_deviation_estimated_means_from_theta = np.round(
    (estimated_means - theta) / theta, decimals=3) * 100

estimated_means = np.round(estimated_means, decimals=3)

Below the estimated mean $M_i$ (red) and the real parameter $\theta_i$ (blue) are plotted for all parameters.

In [ ]:
estimation_mean_parameter_estimation.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.
    evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.
    maximum_likelihood_estimations,
    baseline=theta,
)

In [ ]:
std_parameter_estimation = StdParameterEstimations()

estimated_stds = []
relative_deviation_of_approximate_std_from_estimated_std = []
approximate_std_at_true_parameter = []
for experiment in experiments:
    estimated_std = std_parameter_estimation.calculate(
        estimations_of_parameter=benchmarking.
        maximum_likelihood_estimations[experiment])

    estimated_stds.append(estimated_std)
    diagonal_of_CRLB = statistical_model.calculate_cramer_rao_lower_bound(
        x0=experiment.design, theta=theta).diagonal()

    approximate_std_at_true_parameter.append(np.sqrt(diagonal_of_CRLB))

    relative_deviation_of_approximate_std_from_estimated_std.append(
        np.round((np.sqrt(diagonal_of_CRLB) - estimated_std) / estimated_std,
                 decimals=3) * 100)

estimated_stds = np.round(np.array(estimated_stds), decimals=3)
relative_deviation_of_approximate_std_from_estimated_std = np.array(
    relative_deviation_of_approximate_std_from_estimated_std)

approximate_std_at_true_parameter = np.round(
    np.array(approximate_std_at_true_parameter), decimals=3)

Below the estimated standard deviation $\sqrt{V_i}$ (red) and $\hat{\sigma}_{i}=\sqrt{I^{-1}(\theta_{true})_{ii}}$ (blue) are plotted for all parameters.

In [ ]:
baseline = np.sqrt(
    np.array([
        statistical_model.calculate_cramer_rao_lower_bound(
            x0=design.design, theta=theta).diagonal() for design in experiments
    ]).T)

std_parameter_estimation.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.maximum_likelihood_estimations,
    baseline=baseline,
)

The results are shown in the below data frame.
Recall the true parameter to be $\theta_{true}=(4,2300,0.8)$. 

In [ ]:
data = []

index = [experiment.name for experiment in experiments]

columns = pd.MultiIndex.from_product([[f"i={i}" for i in range(3)],
                                      [
                                          "M_i",
                                          "M_{rel,i}",
                                          "\sqrt{V_i}",
                                          "\theta_{i}",
                                          "\theta_{rel,i}",
]])

for parameter_index, _ in enumerate(theta):
    data = data + [
        estimated_means.T[parameter_index],
        relative_deviation_estimated_means_from_theta.T[parameter_index],
        estimated_stds.T[parameter_index],
        approximate_std_at_true_parameter.T[parameter_index],
        relative_deviation_of_approximate_std_from_estimated_std.
        T[parameter_index],
    ]

data = np.array(data).T

benchmark_table = pd.DataFrame(
    data=data,
    index=index,
    columns=columns,
)
benchmark_table

Furthermore, we plot the determinants of the FIM of the respective experiments (red). The blue line symbolizes the highest determinant.

In [ ]:
det_metric = DeterminantOfFisherInformationMatrix(
    theta=theta, statistical_model=statistical_model)

fig = det_metric.plot(
    evaluations_blackbox_function_for_each_design=benchmarking.
    evaluations_blackbox_function,
    estimations_of_parameter_for_each_design=benchmarking.
    maximum_likelihood_estimations,
    baseline="max",
)
fig.show()

## Histograms of the estimated parameter vector component distributions

In [ ]:
MLEs_pi_experiment = benchmarking.maximum_likelihood_estimations[pi_design]

diagonal_CRLB_pi_experiment = statistical_model.calculate_cramer_rao_lower_bound(
    x0=pi_design.design, theta=theta).diagonal()

In [ ]:
for index in range(3):
    data = [
        go.Histogram(
            x=MLEs_pi_experiment.T[index].T,
            histnorm='probability density',
            name="MLE",
        )
    ]
    fig = styled_figure_latex(data=data,
                              title_x=fr"$\theta_{index}$",
                              title_y=r"$\rho$")

    fig.add_trace(
        normal_distribution(x_range=np.arange(0.6 * theta[index],
                                              1.4 * theta[index], 0.005),
                            mu=theta[index],
                            sigma=np.sqrt(diagonal_CRLB_pi_experiment[index]),
                            name_dist="CRLB"))
    fig.show()

In [ ]:
print(benchmark_table.to_latex())